In [1]:
conda activate TFgpu


Note: you may need to restart the kernel to use updated packages.


<div align="center">
  <img src="DisplayIMG/Cartoon.png" alt="Image Description" />
</div>

In [ ]:
# import cv2
# import os
# import numpy as np
# from PIL import Image
# import pandas as pd
# from sklearn import preprocessing
# from sklearn.model_selection import train_test_split
# from tensorflow.keras.utils import to_categorical
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
# import glob

# # Get the current directory
# current_directory = os.getcwd()

# # Specify the folder name where the images are located
# folder_name = "img"

# # Construct the folder path
# folder_path = os.path.join(current_directory, folder_name)

# # Specify the CSV file path containing image labels
# csv_file_path = os.path.join(current_directory, "labels", "english-clean.csv")

# # Read the CSV file into a DataFrame
# df = pd.read_csv(csv_file_path)

# # Create a dictionary where keys are filenames and values are labels
# labels_dict = df.set_index('image')['label'].to_dict()

# def load_images_from_folder(folder, labels_dict):
#     images = []
#     labels = []
#     for file_path in glob.glob(os.path.join(folder, "*.png")):
#         img = Image.open(file_path)
#         if img is not None:
#             img = img.convert('L')  # Convert image to grayscale
#             img = img.resize((28, 28))  # Resize the image
#             np_img = np.array(img)
#             images.append(np_img)
#             filename = os.path.basename(file_path)
#             label = labels_dict[filename]  # Get the label from the filename
#             labels.append(label)
#     return images, labels

# images, labels = load_images_from_folder(folder_path, labels_dict)

# le = preprocessing.LabelEncoder()
# numerical_labels = le.fit_transform(labels)  # Convert labels to integers

# images = np.array(images)  # Convert list of arrays to a single array
# images = images / 255.0  # Normalize pixel values
# images = images.reshape(-1, 28, 28, 1)  # Reshape array for CNN

# encoded_labels = to_categorical(numerical_labels)  # One-hot encode labels

# X_train, X_test, y_train, y_test = train_test_split(images, encoded_labels, test_size=0.2, random_state=42, shuffle=True)

# model = Sequential()
# model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(Flatten())
# model.add(Dense(64, activation='relu'))
# num_classes = len(set(numerical_labels))
# model.add(Dense(num_classes, activation='softmax'))

# model.compile(optimizer='adam',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

# model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))


Epoch 1/5
86/86 [==============================] - 1s 6ms/step - loss: 4.1225 - accuracy: 0.0238 - val_loss: 4.0857 - val_accuracy: 0.0220
Epoch 2/5
86/86 [==============================] - 0s 4ms/step - loss: 3.7175 - accuracy: 0.0869 - val_loss: 3.2641 - val_accuracy: 0.1745
Epoch 3/5
86/86 [==============================] - 0s 4ms/step - loss: 2.7254 - accuracy: 0.2819 - val_loss: 2.5832 - val_accuracy: 0.3402
Epoch 4/5
86/86 [==============================] - 0s 6ms/step - loss: 2.1166 - accuracy: 0.4249 - val_loss: 2.2214 - val_accuracy: 0.4223
Epoch 5/5
86/86 [==============================] - 0s 4ms/step - loss: 1.7076 - accuracy: 0.5246 - val_loss: 1.8346 - val_accuracy: 0.5337


In [ ]:
import cv2
import os
import numpy as np
from PIL import Image
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
import glob
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report

# Get the current directory
current_directory = os.getcwd()

# Specify the folder name where the images are located
folder_name = "img"

# Construct the folder path
folder_path = os.path.join(current_directory, folder_name)

# Specify the CSV file path containing image labels
csv_file_path = os.path.join(current_directory, "labels", "english-clean.csv")

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Create a dictionary where keys are filenames and values are labels
labels_dict = df.set_index('image')['label'].to_dict()

def load_images_from_folder(folder, labels_dict):
    images = []
    labels = []
    for file_path in glob.glob(os.path.join(folder, "*.png")):
        img = Image.open(file_path)
        if img is not None:
            img = img.convert('L')  # Convert image to grayscale
            img = img.resize((28, 28))  # Resize the image
            np_img = np.array(img)
            images.append(np_img)
            filename = os.path.basename(file_path)
            label = labels_dict[filename]  # Get the label from the filename
            labels.append(label)
    return images, labels

images, labels = load_images_from_folder(folder_path, labels_dict)

le = preprocessing.LabelEncoder()
numerical_labels = le.fit_transform(labels)  # Convert labels to integers

images = np.array(images)  # Convert list of arrays to a single array
images = images / 255.0  # Normalize pixel values
images = images.reshape(-1, 28, 28, 1)  # Reshape array for CNN

encoded_labels = to_categorical(numerical_labels)  # One-hot encode labels

X_train, X_test, y_train, y_test = train_test_split(images, encoded_labels, test_size=0.2, random_state=42, shuffle=True)



model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.15))  # Adjusted Dropout

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.3))  # Adjusted Dropout

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))  # Adjusted Dropout

num_classes = len(set(numerical_labels))
model.add(Dense(num_classes, activation='softmax'))

# Changed optimizer to SGD
opt = SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# # Data augmentation
# datagen = ImageDataGenerator(
#     rotation_range=10,
#     zoom_range = 0.1,
#     width_shift_range=0.1,
#     height_shift_range=0.1)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=30,
    zoom_range = 0.05,
    width_shift_range=0.05,
    height_shift_range=0.05)

datagen.fit(X_train)

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=1500)


# Model Checkpoint
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)

# Fit the model
model.fit(datagen.flow(X_train, y_train, batch_size=32),
          epochs=10000,
          validation_data=(X_test, y_test),
          callbacks=[early_stopping])

history = model.fit(X_train, y_train)

# 86/86 [==============================] - 0s 3ms/step - loss: 0.0439 - accuracy: 0.9813
# Train Loss: 0.043908171355724335
# Train Accuracy: 0.9813050031661987
    
# # 22/22 [==============================] - 0s 3ms/step - loss: 0.3703 - accuracy: 0.8695
# # Test Loss: 0.3703301250934601
# # Test Accuracy: 0.8695014715194702
# # 22/22 [==============================] - 0s 3ms/step  

# 22/22 [==============================] - 0s 3ms/step - loss: 0.4984 - accuracy: 0.8636
# Test Loss: 0.49838370084762573
# Test Accuracy: 0.8636363744735718
# 22/22 [==============================] - 0s 3ms/step
# Classification Report:
    
# Epoch 1608/7000

Epoch 1/10000
86/86 [==============================] - 2s 14ms/step - loss: 4.1167 - accuracy: 0.0883 - val_loss: 4.5943 - val_accuracy: 0.0147
Epoch 2/10000
86/86 [==============================] - 1s 10ms/step - loss: 3.0705 - accuracy: 0.2353 - val_loss: 4.9655 - val_accuracy: 0.0587
Epoch 3/10000
86/86 [==============================] - 1s 12ms/step - loss: 2.4483 - accuracy: 0.3574 - val_loss: 5.0304 - val_accuracy: 0.0733
Epoch 4/10000
86/86 [==============================] - 1s 10ms/step - loss: 2.1158 - accuracy: 0.4219 - val_loss: 5.1795 - val_accuracy: 0.0806
Epoch 5/10000
86/86 [==============================] - 1s 10ms/step - loss: 1.8018 - accuracy: 0.4923 - val_loss: 3.6294 - val_accuracy: 0.2258
Epoch 6/10000
86/86 [==============================] - 1s 10ms/step - loss: 1.6333 - accuracy: 0.5319 - val_loss: 2.3827 - val_accuracy: 0.3944
Epoch 7/10000
86/86 [==============================] - 1s 10ms/step - loss: 1.4235 - accuracy: 0.5733 - val_loss: 3.7169 - val_accuracy:

86/86 [==============================] - 2s 19ms/step - loss: 0.2836 - accuracy: 0.8985 - val_loss: 0.6198 - val_accuracy: 0.8211
Epoch 114/10000
86/86 [==============================] - 1s 12ms/step - loss: 0.2990 - accuracy: 0.8886 - val_loss: 0.9716 - val_accuracy: 0.7346
Epoch 115/10000
86/86 [==============================] - 1s 16ms/step - loss: 0.2967 - accuracy: 0.8955 - val_loss: 0.5231 - val_accuracy: 0.8343
Epoch 116/10000
86/86 [==============================] - 1s 11ms/step - loss: 0.2827 - accuracy: 0.9032 - val_loss: 1.4584 - val_accuracy: 0.6965
Epoch 117/10000
86/86 [==============================] - 1s 10ms/step - loss: 0.2954 - accuracy: 0.8904 - val_loss: 0.4687 - val_accuracy: 0.8490
Epoch 118/10000
86/86 [==============================] - 1s 10ms/step - loss: 0.2818 - accuracy: 0.9014 - val_loss: 0.5312 - val_accuracy: 0.8475
Epoch 119/10000
86/86 [==============================] - 1s 10ms/step - loss: 0.2812 - accuracy: 0.8952 - val_loss: 0.5162 - val_accuracy: 0

86/86 [==============================] - 1s 13ms/step - loss: 0.2037 - accuracy: 0.9216 - val_loss: 0.5229 - val_accuracy: 0.8416
Epoch 226/10000
86/86 [==============================] - 1s 10ms/step - loss: 0.1944 - accuracy: 0.9238 - val_loss: 0.6270 - val_accuracy: 0.8284
Epoch 227/10000
86/86 [==============================] - 1s 11ms/step - loss: 0.1970 - accuracy: 0.9252 - val_loss: 1.3351 - val_accuracy: 0.7610
Epoch 228/10000
86/86 [==============================] - 1s 11ms/step - loss: 0.2076 - accuracy: 0.9241 - val_loss: 0.4832 - val_accuracy: 0.8592
Epoch 229/10000
86/86 [==============================] - 1s 15ms/step - loss: 0.1940 - accuracy: 0.9263 - val_loss: 0.4695 - val_accuracy: 0.8548
Epoch 230/10000
86/86 [==============================] - 1s 10ms/step - loss: 0.1936 - accuracy: 0.9318 - val_loss: 0.6530 - val_accuracy: 0.8152
Epoch 231/10000
86/86 [==============================] - 1s 12ms/step - loss: 0.1903 - accuracy: 0.9278 - val_loss: 0.5481 - val_accuracy: 0

86/86 [==============================] - 1s 13ms/step - loss: 0.1662 - accuracy: 0.9384 - val_loss: 0.4714 - val_accuracy: 0.8592
Epoch 338/10000
86/86 [==============================] - 1s 12ms/step - loss: 0.1640 - accuracy: 0.9344 - val_loss: 0.4954 - val_accuracy: 0.8519
Epoch 339/10000
86/86 [==============================] - 1s 10ms/step - loss: 0.1734 - accuracy: 0.9377 - val_loss: 0.5041 - val_accuracy: 0.8519
Epoch 340/10000
86/86 [==============================] - 1s 13ms/step - loss: 0.1973 - accuracy: 0.9282 - val_loss: 0.5050 - val_accuracy: 0.8578
Epoch 341/10000
86/86 [==============================] - 1s 11ms/step - loss: 0.1857 - accuracy: 0.9293 - val_loss: 0.4827 - val_accuracy: 0.8402
Epoch 342/10000
86/86 [==============================] - 1s 10ms/step - loss: 0.1704 - accuracy: 0.9373 - val_loss: 0.5343 - val_accuracy: 0.8358
Epoch 343/10000
86/86 [==============================] - 1s 12ms/step - loss: 0.1676 - accuracy: 0.9384 - val_loss: 0.5180 - val_accuracy: 0

86/86 [==============================] - 1s 10ms/step - loss: 0.1371 - accuracy: 0.9468 - val_loss: 0.5132 - val_accuracy: 0.8563
Epoch 450/10000
86/86 [==============================] - 1s 11ms/step - loss: 0.1511 - accuracy: 0.9391 - val_loss: 0.5214 - val_accuracy: 0.8680
Epoch 451/10000
86/86 [==============================] - 1s 10ms/step - loss: 0.1352 - accuracy: 0.9465 - val_loss: 0.5876 - val_accuracy: 0.8431
Epoch 452/10000
86/86 [==============================] - 1s 15ms/step - loss: 0.1490 - accuracy: 0.9439 - val_loss: 0.6272 - val_accuracy: 0.8387
Epoch 453/10000
86/86 [==============================] - 1s 11ms/step - loss: 0.1505 - accuracy: 0.9479 - val_loss: 0.5662 - val_accuracy: 0.8416
Epoch 454/10000
86/86 [==============================] - 1s 10ms/step - loss: 0.1394 - accuracy: 0.9450 - val_loss: 0.5544 - val_accuracy: 0.8548
Epoch 455/10000
86/86 [==============================] - 1s 10ms/step - loss: 0.1481 - accuracy: 0.9424 - val_loss: 0.6391 - val_accuracy: 0

...

Epoch 1401/10000
86/86 [==============================] - 1s 10ms/step - loss: 0.0799 - accuracy: 0.9707 - val_loss: 0.6315 - val_accuracy: 0.8519
Epoch 1402/10000
86/86 [==============================] - 1s 10ms/step - loss: 0.0880 - accuracy: 0.9626 - val_loss: 0.6459 - val_accuracy: 0.8519
Epoch 1403/10000
86/86 [==============================] - 1s 12ms/step - loss: 0.0774 - accuracy: 0.9699 - val_loss: 0.6472 - val_accuracy: 0.8519
Epoch 1404/10000
86/86 [==============================] - 1s 11ms/step - loss: 0.0837 - accuracy: 0.9630 - val_loss: 0.6231 - val_accuracy: 0.8504
Epoch 1405/10000
86/86 [==============================] - 1s 13ms/step - loss: 0.0605 - accuracy: 0.9740 - val_loss: 0.6492 - val_accuracy: 0.8534
Epoch 1406/10000
86/86 [==============================] - 1s 11ms/step - loss: 0.0826 - accuracy: 0.9703 - val_loss: 0.6607 - val_accuracy: 0.8475
Epoch 1407/10000
86/86 [==============================] - 1s 13ms/step - loss: 0.0792 - accuracy: 0.9696 - val_loss: 0

Epoch 1457/10000
86/86 [==============================] - 1s 12ms/step - loss: 0.0904 - accuracy: 0.9644 - val_loss: 0.6771 - val_accuracy: 0.8475
Epoch 1458/10000
86/86 [==============================] - 1s 11ms/step - loss: 0.0959 - accuracy: 0.9619 - val_loss: 0.6715 - val_accuracy: 0.8548
Epoch 1459/10000
86/86 [==============================] - 1s 10ms/step - loss: 0.0950 - accuracy: 0.9659 - val_loss: 0.6775 - val_accuracy: 0.8504
Epoch 1460/10000
86/86 [==============================] - 1s 10ms/step - loss: 0.0792 - accuracy: 0.9659 - val_loss: 0.6762 - val_accuracy: 0.8387
Epoch 1461/10000
86/86 [==============================] - 1s 10ms/step - loss: 0.0741 - accuracy: 0.9740 - val_loss: 0.6604 - val_accuracy: 0.8563
Epoch 1462/10000
86/86 [==============================] - 1s 12ms/step - loss: 0.0855 - accuracy: 0.9677 - val_loss: 0.6733 - val_accuracy: 0.8519
Epoch 1463/10000
86/86 [==============================] - 1s 11ms/step - loss: 0.0644 - accuracy: 0.9743 - val_loss: 0

Epoch 1513/10000
86/86 [==============================] - 1s 16ms/step - loss: 0.0747 - accuracy: 0.9692 - val_loss: 0.6454 - val_accuracy: 0.8534
Epoch 1514/10000
86/86 [==============================] - 1s 12ms/step - loss: 0.0717 - accuracy: 0.9732 - val_loss: 0.6780 - val_accuracy: 0.8460
Epoch 1515/10000
86/86 [==============================] - 1s 11ms/step - loss: 0.0787 - accuracy: 0.9688 - val_loss: 0.6585 - val_accuracy: 0.8548
Epoch 1516/10000
86/86 [==============================] - 1s 10ms/step - loss: 0.0784 - accuracy: 0.9714 - val_loss: 0.6961 - val_accuracy: 0.8534
Epoch 1517/10000
86/86 [==============================] - 1s 10ms/step - loss: 0.0777 - accuracy: 0.9725 - val_loss: 0.6823 - val_accuracy: 0.8416
Epoch 1518/10000
86/86 [==============================] - 1s 15ms/step - loss: 0.0702 - accuracy: 0.9765 - val_loss: 0.6429 - val_accuracy: 0.8519
Epoch 1519/10000
86/86 [==============================] - 1s 10ms/step - loss: 0.0778 - accuracy: 0.9670 - val_loss: 0

Epoch 1569/10000
86/86 [==============================] - 1s 14ms/step - loss: 0.0849 - accuracy: 0.9699 - val_loss: 0.6533 - val_accuracy: 0.8548
Epoch 1570/10000
86/86 [==============================] - 1s 11ms/step - loss: 0.0652 - accuracy: 0.9732 - val_loss: 0.6456 - val_accuracy: 0.8504
Epoch 1571/10000
86/86 [==============================] - 1s 10ms/step - loss: 0.0714 - accuracy: 0.9754 - val_loss: 0.6624 - val_accuracy: 0.8534
Epoch 1572/10000
86/86 [==============================] - 1s 11ms/step - loss: 0.0825 - accuracy: 0.9641 - val_loss: 0.6330 - val_accuracy: 0.8431
Epoch 1573/10000
86/86 [==============================] - 1s 13ms/step - loss: 0.0763 - accuracy: 0.9729 - val_loss: 0.6889 - val_accuracy: 0.8460
Epoch 1574/10000
86/86 [==============================] - 1s 13ms/step - loss: 0.0860 - accuracy: 0.9696 - val_loss: 0.6496 - val_accuracy: 0.8475
Epoch 1575/10000
86/86 [==============================] - 1s 12ms/step - loss: 0.0817 - accuracy: 0.9688 - val_loss: 0

Epoch 1625/10000
86/86 [==============================] - 1s 10ms/step - loss: 0.0772 - accuracy: 0.9703 - val_loss: 0.6717 - val_accuracy: 0.8475
Epoch 1626/10000
86/86 [==============================] - 1s 10ms/step - loss: 0.0644 - accuracy: 0.9751 - val_loss: 0.6833 - val_accuracy: 0.8416
Epoch 1627/10000
86/86 [==============================] - 1s 10ms/step - loss: 0.0727 - accuracy: 0.9729 - val_loss: 0.6710 - val_accuracy: 0.8431
Epoch 1628/10000
86/86 [==============================] - 1s 12ms/step - loss: 0.0664 - accuracy: 0.9747 - val_loss: 0.6734 - val_accuracy: 0.8431
Epoch 1629/10000
86/86 [==============================] - 1s 11ms/step - loss: 0.0719 - accuracy: 0.9758 - val_loss: 0.6817 - val_accuracy: 0.8416
Epoch 1630/10000
86/86 [==============================] - 1s 10ms/step - loss: 0.0830 - accuracy: 0.9688 - val_loss: 0.6571 - val_accuracy: 0.8402
Epoch 1631/10000
86/86 [==============================] - 1s 10ms/step - loss: 0.0732 - accuracy: 0.9725 - val_loss: 0

In [ ]:
#Save Model For Furture Use
model.save('EpochTenThousand')

INFO:tensorflow:Assets written to: EpochTenThousand\assets


INFO:tensorflow:Assets written to: EpochTenThousand\assets


In [ ]:
import pickle

# Assume you have a history object from model.fit
# history = model.fit(....)

# Save it under some name
with open('trainHistoryDict', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)


In [ ]:
# with open('trainHistoryDict', 'rb') as file_pi:
#     loaded_history = pickle.load(file_pi)

In [ ]:
# Evaluate the model on the training set
train_loss, train_accuracy = model.evaluate(X_train, y_train)
print("Train Loss:", train_loss)
print("Train Accuracy:", train_accuracy)

86/86 [==============================] - 0s 3ms/step - loss: 0.0132 - accuracy: 0.9974
Train Loss: 0.013188091106712818
Train Accuracy: 0.9974340200424194


In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Make predictions on the test set
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_test, axis=1)

# Print classification report
print("Classification Report:")
print(classification_report(y_true_labels, y_pred_labels, zero_division=1))

# Print confusion matrix
from sklearn.metrics import confusion_matrix
confusion_mtx = confusion_matrix(y_true_labels, y_pred_labels)
print("Confusion Matrix:")
print(confusion_mtx)

22/22 [==============================] - 0s 3ms/step - loss: 0.6097 - accuracy: 0.8504
Test Loss: 0.6097488403320312
Test Accuracy: 0.8504399061203003
22/22 [==============================] - 0s 3ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.36      0.43        14
           1       0.80      0.73      0.76        11
           2       0.91      1.00      0.95        10
           3       1.00      1.00      1.00        20
           4       1.00      1.00      1.00        14
           5       0.93      0.81      0.87        16
           6       0.90      1.00      0.95         9
           7       1.00      0.93      0.97        15
           8       0.83      0.83      0.83        12
           9       0.92      1.00      0.96        11
          10       1.00      1.00      1.00        16
          11       1.00      0.89      0.94         9
          12       0.75      0.67      0.71         9
          13    